In [20]:
import pandas as pd
import random

In [21]:

df_1 = pd.read_csv('../data_limpia/datos_limpios_F1.csv')
df_2 = pd.read_csv('../data_limpia/datos_limpios_F2.csv')
df_4 = pd.read_csv('../data_limpia/datos_limpios_F4.csv')




In [22]:
df_1.rename(columns={'marca': 'MARCA', 'modelo': 'MODELO', 'cilindrada': 'CILINDRADA', 'año': 'AÑO'}, inplace=True)
df_1 

,MARCA,MODELO,CILINDRADA,AÑO
0,TOYOTA,COROLLA,1.6,2005.0
1,TOYOTA,COROLLA,1.6,2006.0
2,TOYOTA,COROLLA,1.6,2007.0
3,TOYOTA,COROLLA,1.6,2008.0
4,TOYOTA,COROLLA,1.6,2009.0
...,...,...,...,...
63,SSANGYONG,NEW ACTYON,2.0,2018.0
64,HYUNDAI,ACCENT,1.6,2006.0
65,HYUNDAI,ACCENT,1.6,2007.0
66,HYUNDAI,ACCENT,1.6,2008.0


In [23]:
df_2

,MARCA,MODELO,CILINDRADA,AÑO
0,CHEVROLET,ASTRA,1.4,1992
1,CHEVROLET,ASTRA,1.4,1993
2,CHEVROLET,ASTRA,1.4,1994
3,CHEVROLET,ASTRA,1.4,1995
4,CHEVROLET,ASTRA,1.4,1996
...,...,...,...,...
566,OPEL,ZAFIRA,2.0,2005
567,OPEL,ZAFIRA,2.0,2006
568,OPEL,ZAFIRA,ND,1999
569,OPEL,ZAFIRA,ND,2000


In [24]:
df_4.rename(columns={'modelo_base_corregido': 'MODELO', 'version_corregida': 'VERSION', 'cilindrada': 'CILINDRADA', 'AÑO_FAB': 'AÑO'}, inplace=True)
df_4.drop(columns=['VERSION'], inplace=True)
df_4.head()

,MARCA,MODELO,CILINDRADA,AÑO
0,HYUNDAI,GRAND I10,NaN,2020
1,SSANGYONG,MUSSO,2.2,2020
2,FORD,EXPLORER,3.5,2020
3,KIA,SOLUTO,1.4,2020
4,RENAULT,OROCH,1.6,2020


In [25]:
df = pd.concat([df_1, df_2, df_4], axis=0, join='outer')
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
df

,MARCA,MODELO,CILINDRADA,AÑO
0,TOYOTA,COROLLA,1.6,2005.0
1,TOYOTA,COROLLA,1.6,2006.0
2,TOYOTA,COROLLA,1.6,2007.0
3,TOYOTA,COROLLA,1.6,2008.0
4,TOYOTA,COROLLA,1.6,2009.0
...,...,...,...,...
677,JAC,S3,1.6,2020
678,GREAT WALL,3,2.0,2020
680,FIAT,UNO,1.4,2020
682,SUZUKI,DZIRE,1.2,2020


In [26]:
import random 

def abreviar_marca(marca, n_variantes=5):
    marca = marca.upper().replace("-", " ")
    tokens = marca.split()
    variantes = set()

    if len(tokens) == 0:
        return []

    p = tokens[0]
    # Tomar primeras 3 letras
    variantes.add(p[:3])
    # Solo consonantes de la primera palabra
    variantes.add("".join([c for c in p if c not in "AEIOU"])[:3])
    # Palabra completa
    variantes.add(p)
    variantes.add(p[:2])
    # Si hay segunda palabra, combinaciones
    if len(tokens) > 1:
        s = tokens[1]
        variantes.add(f"{p[:3]} {s}")
        variantes.add(f"{p[0]}{s}")
        variantes.add(s)
        variantes.add(f"{p} {s}")
        if len(s) > 1:
            variantes.add(f"{p[:2]} {s[:2]}")
        variantes.add(f"{p[:2]}{s[:1]}")

    # Todo junto sin espacios
    variantes.add("".join(tokens))

    # Limitar a n_variantes
    return list(variantes)[:n_variantes]


def abreviar_modelo(modelo, n_variantes=5):
    modelo = modelo.upper().replace("-", " ")
    tokens = modelo.split()
    variantes = set()

    p = tokens[0]
    variantes.add(p[:3])
    variantes.add("".join([c for c in p if c not in "AEIOU"]))
    variantes.add(p)
    variantes.add(p[:2])
    
    if len(tokens) > 1:
        s = tokens[1]
        variantes.add(f"{p[:3]} {s}")
        variantes.add(f"{p[0]}{s}")
        variantes.add(s)
        variantes.add(f"{p} {s}")
        if len(s) > 1:
            variantes.add(f"{p[:2]} {s[:2]}")
        variantes.add(f"{p[:2]}{s[:1]}")

    variantes.add("".join(tokens))
    return list(variantes)

In [27]:
import random

PALABRAS_REPUESTO = ["RADIADOR", "VALVULA", "FARO", "BANDEJA", "AMORTIGUADOR", "ESPEJO", 
                     "PIOLA", "MASCARA", "KIT", "ACEITE", "FRENOS", "DEL", "PARA", "TRASERO", "GEN", "ABSORBEDOR"] + [
                        "RADIADOR", "ROTULA", "BUJE", "FARO", "OPTICO", "MASCARA",
                        "ESPEJO", "AMORT", "PIOLA", "BANDEJA", "ABSORBEDOR", "VALVULA"
                    ]

CONECTORES = ["DEL", "PARA", "GEN", "CON", "TRASERO", "DELANTERO", "COMPATIBLE", "VERSION"]

def generar_frase(marca=None, modelo=None, año=None, cilindrada=None, variantes=5, negativo_ratio=0.2):
    frases = []

    for _ in range(variantes):
        tokens = []
        ner_tags = []

        es_negativo = (marca is None or modelo is None or random.random() < negativo_ratio)

        if es_negativo:
            # Frase negativa sin entidades
            n_palabras = random.randint(3, 6)
            tokens = random.choices(PALABRAS_REPUESTO + CONECTORES, k=n_palabras)
            ner_tags = ["O"] * len(tokens)
            frases.append({"tokens": tokens, "ner_tags": ner_tags})
            continue

        # Frase positiva
        marca_variantes = abreviar_marca(marca, n_variantes=6)
        modelo_variantes = abreviar_modelo(modelo, n_variantes=6)

        if not marca_variantes:
            marca_variantes = [marca.upper()]

        if random.random() < 0.6:
            tokens.append(random.choice(PALABRAS_REPUESTO))
            ner_tags.append("O")

        # Agregar conectores ocasionales antes de la marca
        if random.random() < 0.3:
            tokens.append(random.choice(CONECTORES))
            ner_tags.append("O")

        # Marca
        marca_token = random.choice(marca_variantes)
        marca_tokens = marca_token.split()
        tokens.extend(marca_tokens)
        ner_tags.extend(["B-MARCA"] + ["I-MARCA"]*(len(marca_tokens)-1))

        # Agregar conectores después de la marca a veces
        if random.random() < 0.3:
            tokens.append(random.choice(CONECTORES))
            ner_tags.append("O")

        # Modelos
        n_modelos = 2 if len(modelo_variantes) > 1 and random.random() < 0.5 else 1
        modelos_elegidos = random.sample(modelo_variantes, k=n_modelos)
        for m in modelos_elegidos:
            m_tokens = m.split()
            tokens.extend(m_tokens)
            ner_tags.extend(["B-MODELO"] + ["I-MODELO"]*(len(m_tokens)-1))

        # Posible conector después del modelo
        if random.random() < 0.2:
            tokens.append(random.choice(CONECTORES))
            ner_tags.append("O")

        # Cilindrada
        if cilindrada and random.random() < 0.5:
            c_tokens = str(cilindrada).split()
            tokens.extend(c_tokens)
            ner_tags.extend(["B-CILINDRADA"] + ["I-CILINDRADA"]*(len(c_tokens)-1))

        # Año
        if año and random.random() < 0.5:
            a = año[-2:] if random.random() < 0.5 else año
            a_tokens = str(a).split()
            tokens.extend(a_tokens)
            ner_tags.extend(["B-AÑO"] + ["I-AÑO"]*(len(a_tokens)-1))

        # Otro conector final para simular nombres largos
        if random.random() < 0.2:
            tokens.append(random.choice(CONECTORES))
            ner_tags.append("O")

        # Aleatoriamente mezclar (no recomendado para validación, solo para robustez)
        if random.random() < 0.3:
            combined = list(zip(tokens, ner_tags))
            random.shuffle(combined)
            tokens, ner_tags = zip(*combined)
            tokens = list(tokens)
            ner_tags = list(ner_tags)

        frases.append({"tokens": tokens, "ner_tags": ner_tags})

    return frases


In [28]:
import re

def tokenizar(frase):
    return frase.split()

def anotar_ner(frase_dict, marca, modelo, año=None, cilindrada=None):
    # frase_dict = {"tokens": [...], "ner_tags": [...]}
    tokens = frase_dict["tokens"]
    etiquetas = frase_dict["ner_tags"]
    # Aquí podrías agregar o ajustar etiquetas si quieres (ejemplo: mapear abreviaturas)
    return {"tokens": tokens, "ner_tags": etiquetas}


In [29]:
def crear_dataset(df, frases_por_fila=5):
    dataset = []

    for _, row in df.iterrows():
        marca = row["MARCA"]
        modelo = row["MODELO"]
        año = str(row.get("AÑO", ""))
        cil = str(row.get("CILINDRADA", ""))

        frases = generar_frase(marca, modelo, año, cil, variantes=frases_por_fila)
        
        for frase_dict in frases:
            ejemplo = anotar_ner(frase_dict, marca, modelo, año, cil)
            dataset.append(ejemplo)

    
    return dataset


In [30]:
import json

def guardar_jsonl(dataset, path="dataset_ner.jsonl"):
    with open(path, "w", encoding="utf8") as f:
        for item in dataset:
            json.dump(item, f)
            f.write("\n")


In [31]:
dataset = crear_dataset(df, frases_por_fila=12)
guardar_jsonl(dataset, path="dataset_ner.jsonl")


In [32]:
# Obtener marcas y modelos únicos
marcas_unicas = df["MARCA"].dropna().unique()
modelos_unicos = df["MODELO"].dropna().unique()

In [33]:
# Generar diccionarios con lista de variantes para cada marca y modelo
marcas_var = {m: abreviar_marca(m, n_variantes=6) for m in marcas_unicas}
modelos_var = {m: abreviar_modelo(m, n_variantes=6) for m in modelos_unicos}

In [34]:
# Diccionario modelo a marca (el último año o el primero está bien, solo mapeamos)
modelo_a_marca = df.set_index("MODELO")["MARCA"].to_dict()